In [1]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Dictionary mapping landmark indices to body parts
LANDMARKS = {
    0: "Nose", 1: "Left Eye Inner", 2: "Left Eye", 3: "Left Eye Outer", 4: "Right Eye Inner",
    5: "Right Eye", 6: "Right Eye Outer", 7: "Left Ear", 8: "Right Ear", 9: "Mouth Left",
    10: "Mouth Right", 11: "Left Shoulder", 12: "Right Shoulder", 13: "Left Elbow", 
    14: "Right Elbow", 15: "Left Wrist", 16: "Right Wrist", 17: "Left Pinky", 
    18: "Right Pinky", 19: "Left Index", 20: "Right Index", 21: "Left Thumb", 
    22: "Right Thumb", 23: "Left Hip", 24: "Right Hip", 25: "Left Knee", 
    26: "Right Knee", 27: "Left Ankle", 28: "Right Ankle", 29: "Left Heel", 
    30: "Right Heel", 31: "Left Foot Index", 32: "Right Foot Index"
}

# Desired video frame size (width, height)
frame_width, frame_height = 1280, 720

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

# Set camera resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

# Set up the Pose model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    start_time = time.time()
    frame_count = 0
    show_labels = True  # Toggle for showing labels

    # Create a named window and set its size
    cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Pose Detection', frame_width, frame_height)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Ignoring empty camera frame.")
            break

        # Resize the frame to the desired size
        frame = cv2.resize(frame, (frame_width, frame_height))

        # Convert the image from BGR to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False  # Improve performance

        # Process the image to detect poses
        results = pose.process(image_rgb)

        # Convert the image back to BGR for OpenCV
        image_rgb.flags.writeable = True
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Draw the pose annotations on the image
        if results.pose_landmarks:
            # Draw connections and landmarks
            mp_drawing.draw_landmarks(
                image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )

            if show_labels:
                # Draw landmarks with labels next to each landmark
                for idx, landmark in enumerate(results.pose_landmarks.landmark):
                    if idx in LANDMARKS:
                        h, w, _ = image_bgr.shape
                        cx, cy = int(landmark.x * w), int(landmark.y * h)

                        # Position the label slightly above the landmark
                        label_pos = (cx, cy - 15 if cy - 15 > 0 else cy + 15)

                        # Draw a filled rectangle as a background for the text
                        text_size, _ = cv2.getTextSize(LANDMARKS[idx], cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                        text_w, text_h = text_size
                        cv2.rectangle(image_bgr, (label_pos[0] - 5, label_pos[1] - text_h - 10), 
                                      (label_pos[0] + text_w + 5, label_pos[1]), (0, 0, 0), cv2.FILLED)

                        # Draw the text label
                        cv2.putText(image_bgr, LANDMARKS[idx], (label_pos[0], label_pos[1] - 5), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # Display the image with pose annotations
        cv2.imshow('Pose Detection', image_bgr)

        # Calculate and display FPS
        frame_count += 1
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            fps = frame_count / elapsed_time
            cv2.putText(image_bgr, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Take snapshot if 's' is pressed
        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):
            snapshot_filename = f'snapshot_{int(time.time())}.png'
            cv2.imwrite(snapshot_filename, image_bgr)
            print(f"Snapshot saved as {snapshot_filename}")

        # Break the loop if 'q' is pressed
        if key & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


c:\Users\Dell\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
